In [1]:
import os
import pandas as pd
from langchain_groq.chat_models import ChatGroq
from langchain.prompts import ChatPromptTemplate
from pprint import pprint
from dotenv import load_dotenv
load_dotenv()

df = pd.read_csv("products_2000.csv", nrows=10, usecols=["title", "description"])
df.head()

c:\Users\fredr\OneDrive\Desktop\llm-utilities\llm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


,title,description
0,Nike Footwear Max 410,Nike Footwear Max 410: High-quality Footwear /...
1,Nike Footwear X 3612,Nike Footwear X 3612: High-quality Footwear / ...
2,Nike Footwear Max 6202,Nike Footwear Max 6202: High-quality Footwear ...
3,Nike Footwear Classic 8752,Nike Footwear Classic 8752: High-quality Footw...
4,Nike Footwear 2025 6483,Nike Footwear 2025 6483: High-quality Footwear...


**LLMChain**

In [2]:
from langchain.chains import LLMChain


llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model="openai/gpt-oss-120b", temperature=0.9)
prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}")

chain = LLMChain(llm=llm, prompt=prompt)
product = "Running Shoes"
print(chain.run(product=product))

C:\Users\fredr\AppData\Local\Temp\ipykernel_3928\351539572.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)
C:\Users\fredr\AppData\Local\Temp\ipykernel_3928\351539572.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(chain.run(product=product))


Below are a handful of name ideas — and a quick checklist you can use to decide which one fits your vision best. Feel free to mix‑and‑match words, add a twist, or use the list as a springboard for your own brainstorming.

---

## 1️⃣  Ready‑to‑Use Names

| # | Name | Why It Works |
|---|------|--------------|
| 1 | **StrideForce** | Conveys power and forward motion; “stride” is instantly linked to running. |
| 2 | **RunVantage** | Suggests a competitive edge (“advantage”) for runners. |
| 3 | **Velocity Sole** | Highlights speed while reminding shoppers that it’s a shoe. |
| 4 | **PulseFoot** | “Pulse” evokes heart‑rate, energy, and rhythm—key to running. |
| 5 | **AeroStride** | “Aero” hints at lightweight, breathable tech. |
| 6 | **Kinetic Kicks** | Fun, memorable, and emphasizes movement. |
| 7 | **TrailBlaze Performance** | Works if you target both road and trail runners; “blaze” = fast. |
| 8 | **Momentum Gear** | Implies continuous forward motion—exactly what runners want. |
| 9

**SequentialChain**

1. SimpleSequentialChain - single input/output
2. SequentialChain - multiple inputs/outputs

In [3]:
from langchain.chains import SimpleSequentialChain
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"], model="openai/gpt-oss-120b", temperature=0.9)


first_prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}? Answer in one word.")
chain_one = LLMChain(llm=llm, prompt=first_prompt)

second_prompt = ChatPromptTemplate.from_template("Write a 20 word description for the following company: {company_name}")
chain_two = LLMChain(llm=llm, prompt=second_prompt)

overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
Stride
Stride empowers businesses with AI-driven analytics, delivering real-time insights, scalable solutions, and seamless integration for sustainable growth across global markets.

> Finished chain.


'Stride empowers businesses with AI-driven analytics, delivering real-time insights, scalable solutions, and seamless integration for sustainable growth across global markets.'

In [4]:
from langchain.chains import SequentialChain

llm = ChatGroq(api_key=os.environ['GROQ_API_KEY'], temperature=0.0, model="openai/gpt-oss-120b")

first_prompt = ChatPromptTemplate.from_template("Translate the following review to English: \n\n{Review}")
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="EnglishReview")

second_prompt = ChatPromptTemplate.from_template("Can you summarize the following review in 1 sentence: \n\n{EnglishReview}")
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template("What language is the following review: \n\n{Review}")
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following summary in the specified language: \nSummary: {summary}\n\nLanguage: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")


review = "Runwear - отличная компания, создающая высококачественную спортивную одежду для бегунов. Их продукция сочетает в себе передовые технологии и стильный дизайн. Одежда комфортная, дышащая и долговечная, идеально подходит для интенсивных тренировок. Runwear мотивирует и помогает достигать лучших результатов. Рекомендую всем, кто ценит комфорт и производительность!"

overall_sequential_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["followup_message"],
    verbose=True
)
overall_sequential_chain.run(Review=review)



> Entering new SequentialChain chain...

> Finished chain.


'Отлично! Ваше описание уже подчёркивает ключевые преимущества Runwear, но позвольте добавить несколько деталей, которые могут ещё больше убедить потенциальных покупателей.\n\nRunwear действительно сочетает в себе высокое качество материалов и стильный дизайн, что делает их одежду не только функциональной, но и визуально привлекательной. Технология «breathable mesh» обеспечивает оптимальную вентиляцию, позволяя коже «дышать» даже во время интенсивных тренировок в жаркую погоду. Благодаря использованию износостойких волокон, изделия сохраняют форму и цвет после многочисленных стирок, что особенно важно для тех, кто бегает регулярно.\n\nКроме того, продуманные крои и эластичные швы снижают риск натираний и обеспечивают свободу движений, что напрямую влияет на комфорт и эффективность тренировок. Многие спортсмены отмечают, что наличие ярких, но не кричащих цветов повышает мотивацию и делает процесс бега более приятным.\n\nЕсли подытожить, Runwear — это не просто спортивная одежда, а надёж

In [5]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate
import templates

c:\Users\fredr\OneDrive\Desktop\llm-utilities\langchain\templates.py:48: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """


### Destination Chain

In [6]:
llm = ChatGroq(api_key=os.environ['GROQ_API_KEY'], temperature=0.0, model="llama-3.3-70b-versatile")

destination_chains = {}

for p_info in templates.prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in templates.prompt_infos]
destinations_str = "\n".join(destinations)
pprint(destinations_str)

('Physics_Chain: Good for answering questions about classical and modern '
 'physics, including mechanics, electromagnetism, quantum theory, relativity, '
 'and thermodynamics.\n'
 'Math_Chain: Ideal for solving and explaining mathematical problems across '
 'algebra, calculus, geometry, number theory, and more abstract areas.\n'
 'History_Chain: Useful for exploring historical events, timelines, causes and '
 'consequences, and broader thematic connections across world history.\n'
 'Computerscience_Chain: Great for programming help, understanding algorithms, '
 'system design, AI/ML topics, and theoretical computer science problems.')


### Router Chain

In [7]:
router_template = templates.MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template, input_variables=['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

### Default Chain

In [8]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [11]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

pprint(chain.run("What is black body radiation? Explain with 4 sentences"), width=140)



> Entering new MultiPromptChain chain...
Physics_Chain: {'input': 'Explain black body radiation in 4 sentences'}
> Finished chain.
('Black body radiation refers to the thermal electromagnetic radiation emitted by an object at a certain temperature, with the object '
 'being a perfect absorber of all incident radiation, hence the term "black body". As the temperature of the black body increases, the '
 'radiation emitted spans a wide range of wavelengths, from radio waves to gamma rays, with the peak wavelength shifting to shorter '
 "wavelengths according to Wien's displacement law. The distribution of radiation intensity versus wavelength is described by Planck's "
 'law, which introduced the concept of quantized energy and laid the foundation for quantum mechanics. At different temperatures, the '
 'black body radiation spectrum changes, with examples including the red glow of heated metal, the yellow light of the sun, and the cosmic '
 'microwave background radiation, which is the

In [10]:
pprint(chain.run("Why does every cell in our body contain DNA? Explain with 3 sentences"), width=140)



> Entering new MultiPromptChain chain...
None: {'input': 'Why does every cell in the human body contain DNA? Explain the role of DNA in cells with 3 sentences.'}
> Finished chain.
('Every cell in the human body contains DNA because it serves as the genetic blueprint for the cell, providing instructions for growth, '
 'development, and function. The role of DNA in cells is to store and transmit genetic information, allowing cells to produce proteins, '
 'replicate, and respond to their environment. By containing DNA, cells can access the necessary genetic instructions to perform their '
 'specific functions, maintain cellular processes, and contribute to the overall health and development of the individual.')
